# Modelling for Data B (v.1.0)

Brief description of the notebook

# Goal

Goal of the notebook

# Plan

Checklist what will be done on the notebook :

    [ ] Get Data
    [ ] 

# Summary

Summary result of the notebook


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from gensim.models import KeyedVectors
import numpy as np

import torch
from torch.autograd import Variable
import random
import matplotlib.pyplot as plt
import torch.utils.data as utils_data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
RAW_PATH="../data/raw/"
INTERIM_PATH="../data/interim/"
PROCESSED_PATH="../data/processed/"
MODEL_PATH="../models/"

## Get Data

In [3]:
data_raw=pd.read_csv(PROCESSED_PATH+'20190902_data_clean_B.csv')
wv = KeyedVectors.load(MODEL_PATH+"embedding_B.wv", mmap='r')

/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
wv.vector_size

75

In [5]:
def word_embbed(w):
    vec=([0.0]*wv.vector_size)
    try:
        vec=wv[w].tolist()
    except:
        pass
    return np.array(vec)

def list_word_embbed(list_words):
    embbed=[word_embbed(w) for w in list_words]
    return np.asarray(embbed)

In [6]:
# wv['kesulitan']

In [7]:
data_raw.sample(10)

,RES_ID,RESPONSE,LABEL,RESPONSE_CLN,RESPONSE_CLN2,count_word,count_word_cln,count_word_cln2,avg_word_length,avg_word_length_cln,...,is_layak,is_pakaian,is_konsumen,is_pahala,is_kondisi,is_membutuhkan,is_karena,count_stopwords,count_topwords,RESPOND_USED
81,TRB82,untuk mengurangi penggunan air,0,mengurangi penggunan air,untuk mengurangi penggunan,4,3,3,6.750000,7.333333,...,0,0,0,0,0,0,0,1,0,untuk mengurangi penggunan 0 0 0 0 0 0 0 0 0 0...
28,TRB29,karena t sebagai upaya untuk membuat produksi ...,1,upaya produksi pakaian beretika,karena sebagai upaya untuk membuat produksi pa...,11,4,10,6.090909,7.000000,...,0,1,0,0,0,0,1,6,3,karena sebagai upaya untuk membuat produksi pa...
169,TRB170,mungkin pakaian itu tidak bagus untuk di pakai...,0,pakaian bagus pakai pakaian muat pakai orang,mungkin pakaian tidak bagus untuk pakai lagi m...,20,7,16,5.100000,5.428571,...,0,1,0,0,0,0,0,11,3,mungkin pakaian tidak bagus untuk pakai lagi m...
97,TRB98,karena jika ikhlas menyumangkan sedikit uang k...,0,ikhlas menyumangkan uang anak yatim panti asuh...,karena jika ikhlas menyumangkan sedikit uang k...,20,9,19,5.600000,6.222222,...,0,0,0,1,0,0,1,11,1,karena jika ikhlas menyumangkan sedikit uang k...
236,TRB237,karna harga yang ditawarkan tidak sebanding de...,0,harga ditawarkan sebanding kualitas kaus konsu...,karna harga yang ditawarkan tidak sebanding de...,30,12,29,5.800000,7.000000,...,0,0,1,0,0,0,1,17,1,karna harga yang ditawarkan tidak sebanding de...
162,TRB163,karana dengan menyumbang kita bisa membuat pro...,0,menyumbang produksi pakain beretika,karana dengan menyumbang kita bisa membuat pro...,11,4,11,6.454545,8.000000,...,0,0,0,0,0,0,1,7,3,karana dengan menyumbang kita bisa membuat pro...
127,TRB128,karena konsumen ingin berupaya untuk membuat p...,0,konsumen berupaya produksi menjad beretika,karena konsumen ingin berupaya untuk membuat p...,10,5,10,6.600000,7.600000,...,0,0,1,0,0,0,1,5,3,karena konsumen ingin berupaya untuk membuat p...
194,TRB195,karena dengan menyumbang maka akan membuat pro...,1,menyumbang produksi pakaian beretika kondisi k...,karena dengan menyumbang maka akan membuat pro...,18,7,17,6.388889,7.285714,...,0,1,0,0,1,0,1,11,7,karena dengan menyumbang maka akan membuat pro...
231,TRB232,karena masyarakat menginginkan pakaian yang be...,1,masyarakat pakaian beretika upah kerja layak,karena masyarakat menginginkan pakaian yang be...,14,6,12,6.357143,6.500000,...,1,1,0,0,0,0,1,8,4,karena masyarakat menginginkan pakaian yang be...
292,TRB293,karna menyumbang itu adalah hal positif dan me...,0,menyumbang positif amal ibadah,karna menyumbang adalah positif mendapatkan am...,10,4,7,5.800000,6.750000,...,0,0,0,0,0,0,1,6,1,karna menyumbang adalah positif mendapatkan am...


In [8]:
col1=[c for c in data_raw.columns if ('is_' in c)|('avg_' in c)|('count_' in c)]
col2=['RESPOND_USED']

In [9]:
len(col1)

25

In [10]:
col1

['count_word',
 'count_word_cln',
 'count_word_cln2',
 'avg_word_length',
 'avg_word_length_cln',
 'is_sumbang',
 'is_kerja',
 'is_upaya',
 'is_memilih',
 'is_pabrik',
 'is_membantu',
 'is_orang',
 'is_pekerja',
 'is_produksi',
 'is_menyumbang',
 'is_beretika',
 'is_layak',
 'is_pakaian',
 'is_konsumen',
 'is_pahala',
 'is_kondisi',
 'is_membutuhkan',
 'is_karena',
 'count_stopwords',
 'count_topwords']

## Data Preparation

In [11]:
data_raw.shape

(305, 31)

In [12]:
## Split Data
data_raw_tr=data_raw.sample(250,random_state=231)
data_raw_te=data_raw[~(data_raw.RES_ID.isin(data_raw_tr.RES_ID.unique()))]

In [13]:
X1_tr=data_raw_tr[col1].as_matrix()
X2_tr=np.array([list_word_embbed(i[0].split(' ')) for i in data_raw_tr[col2].as_matrix()])

X1_te=data_raw_te[col1].as_matrix()
X2_te=np.array([list_word_embbed(i[0].split(' ')) for i in data_raw_te[col2].as_matrix()])

X_tr=[(Variable(torch.from_numpy(z1).type('torch.DoubleTensor')),Variable(torch.from_numpy(z2).type('torch.DoubleTensor'))) for z1,z2 in zip(X1_tr,X2_tr)]
X_te=[(Variable(torch.from_numpy(z1).type('torch.DoubleTensor')),Variable(torch.from_numpy(z2).type('torch.DoubleTensor'))) for z1,z2 in zip(X1_te,X2_te)]

Y_tr=data_raw_tr['LABEL'].as_matrix()
Y_te=data_raw_te['LABEL'].as_matrix()

Y_tr=Y_tr.reshape(len(Y_tr),1)
Y_te=Y_te.reshape(len(Y_te),1)

encoder = OneHotEncoder()
encoder.fit(Y_tr)

Y_tr = encoder.transform(Y_tr)
Y_te = encoder.transform(Y_te)

/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda

In [14]:
X_tr[0][1].size()

torch.Size([15, 75])

In [15]:
X_tr[0][0].size()

torch.Size([25])

## Prepare Model

In [92]:
## LSTM Example
class ModelB(torch.nn.Module):
    def __init__(self, input_size1, input_size2, output_size):        
        super(ModelB, self).__init__()
        self.input_size1=input_size1
        self.input_size2=input_size2
        
        self.softmax = torch.nn.Softmax()
        
        self.features1 = torch.nn.Sequential(
            torch.nn.LSTM(self.input_size1, 150)
        )
        
        self.features2 = torch.nn.Sequential(
            torch.nn.Linear(self.input_size2, 50,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(50, 25,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(25, 5,bias=False),
            torch.nn.ReLU(inplace=True)
        )
        
        self.features3 = torch.nn.Sequential(
            torch.nn.Linear(30, 30,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(30, 30,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(30, 15,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(15, 10,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(10, 2,bias=False),
            torch.nn.Softmax())
        
        
        self.features4 = torch.nn.Sequential(
            torch.nn.Linear(150, 50,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(50, 50,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(50, 25,bias=False),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(25, 25,bias=False),
            torch.nn.ReLU(inplace=True)
        )
        
    def forward(self, input1,input2):
        torch.manual_seed(193)
        # input1=input[1]
        # input2=input[0]

        input1=input1.view(1,15, -1)
        input2=input2.view(1,-1)

        x1,h = self.features1(input1)
        x1_1=x1[:,-1]
        x1 = self.features4(x1_1)
        x2 = self.features2(input2)
        
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        
        out=torch.cat((x1, x2), 1)
        y_pred=self.features3(out)
        
        return y_pred,'-'

In [93]:
EMBEDDING_LEN=wv.vector_size

model1_B = ModelB(input_size1=EMBEDDING_LEN,
                  input_size2=25, 
                  output_size=1)
model1_B=model1_B.double()

In [94]:
learningRate=0.0007
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCELoss()
# criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model1_B.parameters(),lr=learningRate)

In [95]:
# model1_A(X_tr[1][1],X_tr[1][0])

## Train the model

In [96]:
# X_tr0, X_tr1, Y_tr0, Y_tr1 = train_test_split(X_tr, Y_tr, test_size=0.15, random_state=113)

In [97]:
# len(X_tr0)

In [98]:
epochs=20
for epoch in range(epochs):
    model1_B.train()
    # Converting inputs and labels to Variable
    running_loss_train = 0
    batch_idx=1
    for (data, target) in zip(X_tr,Y_tr):
        inputs = data
        labels = Variable(torch.from_numpy(target.toarray()))
        # labels = Variable(torch.from_numpy(target.toarray()).type('torch.DoubleTensor'))
        optimizer.zero_grad()
        outputs, hidden = model1_B(inputs[1],inputs[0])
        # get loss for the predicted output
        loss = criterion(outputs, labels)
        loss.backward()
        # update parameters
        optimizer.step()

        running_loss_train += (loss.item() - running_loss_train) / batch_idx
        batch_idx+=1
    
    model1_B.eval()
    batch_idx=1
    with torch.no_grad():
        running_loss_test = 0
        for (data, target) in zip(X_te,Y_te):
            inputs_te = data
            labels_te = Variable(torch.from_numpy(target.toarray()))
            outputs_te, hidden = model1_B(inputs_te[1],inputs_te[0])
            loss_te = criterion(outputs_te, labels_te)
            running_loss_test += (loss_te.item() - running_loss_test) / batch_idx
            batch_idx+=1
    
    if epoch % 2 == 0:
        print(outputs.data.numpy(),labels.data.numpy())
        print("running loss train", running_loss_train)
        print("running loss test", running_loss_test)

[[0.49027039 0.50972961]] [[0. 1.]]
running loss train 0.6924366904578994
running loss test 0.6815489942232543
[[0.39736505 0.60263495]] [[0. 1.]]
running loss train 0.6748009962292006
running loss test 0.6184732678817116
[[0.34450988 0.65549012]] [[0. 1.]]
running loss train 0.6319281677678042
running loss test 0.654693387513263
[[0.19825279 0.80174721]] [[0. 1.]]
running loss train 0.5841971758166583
running loss test 0.5918186694235245
[[0.18213609 0.81786391]] [[0. 1.]]
running loss train 0.5673429227199828
running loss test 0.6089888243305364
[[0.17473336 0.82526664]] [[0. 1.]]
running loss train 0.5601814834034544
running loss test 0.6285444335195104
[[0.16019593 0.83980407]] [[0. 1.]]
running loss train 0.5526964148886383
running loss test 0.6100187370429658
[[0.17259021 0.82740979]] [[0. 1.]]
running loss train 0.5574886919979232
running loss test 0.6263567649504755
[[0.16501646 0.83498354]] [[0. 1.]]
running loss train 0.5550767969617703
running loss test 0.6225784164916837
[[

In [99]:
model1_B.eval()

ModelB(
  (softmax): Softmax(dim=None)
  (features1): Sequential(
    (0): LSTM(75, 150)
  )
  (features2): Sequential(
    (0): Linear(in_features=25, out_features=50, bias=False)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=50, out_features=25, bias=False)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=25, out_features=5, bias=False)
    (5): ReLU(inplace=True)
  )
  (features3): Sequential(
    (0): Linear(in_features=30, out_features=30, bias=False)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=30, bias=False)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=30, out_features=15, bias=False)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=15, out_features=10, bias=False)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=10, out_features=2, bias=False)
    (9): Softmax(dim=None)
  )
  (features4): Sequential(
    (0): Linear(in_features=150, out_features=50, bias=False)
    (1): ReLU(inplace=True)
    (2): Linear(in_

## Evaluation

In [118]:
def evaluation_model(model,X,Y):
    yt=[]
    yp=[]
    for (data,y) in zip(X,Y.toarray()):
        outputs, hidden = model(data[1],data[0])
        yp.append(outputs.data.numpy()[-1][-1])
        yt.append(y[-1])

    yp=(np.array(yp)>0.4).astype(float).tolist()

    tn, fp, fn, tp = confusion_matrix(yt,yp).ravel()
    acc=(tn+tp)/(tn+fp+fn+tp)
    prec=tp/(tp+fp)
    recc=tp/(tp+fn)

    print('Accuracy :',acc )
    print('Precision :',prec )
    print('Recall :',recc )

In [119]:
evaluation_model(model1_B,X_tr,Y_tr)

Accuracy : 0.736
Precision : 0.7169811320754716
Recall : 0.8444444444444444


In [120]:
evaluation_model(model1_B,X_te,Y_te)

Accuracy : 0.7090909090909091
Precision : 0.7428571428571429
Recall : 0.7878787878787878


In [122]:
torch.save(model1_B, MODEL_PATH+"20190903_model_B_1.pt")

## Conclusions